# Neural Style Transfer

[Original paper](https://arxiv.org/abs/1508.06576) by Gatys et al.

## Math

### Content and Style

Let $C_{nn}$ be a pre-trained network. $C_{nn}(X)$ is the network fed by $X$. Define $F_{XL} \in C_{nn}(X)$ as the feature maps of $C_{nn}(X)$ at layer $L$.

We define the content of $X$ at layer $L$ simply be $F_{XL}$. So with another picture $Y$, the distance of **content** between $X$ and $Y$ is defined as:

$$ D_C^L(X,Y) = ||F_{XL} - F_{YL}||^2 = \sum_{i}(F_{XL}(i) - F_{YL}(i))^2 $$

Then, we define the style of the image $X$ at layer $L$ as the gram matrix of $F_{XL}$. That is

$$ G_{XL}(k, l) = \langle F_{XL}^k, F_{XL}^l\rangle = \sum_{i}F_{XL}^k(i) \cdot F_{YL}^l(i)$$

So we can define the distance of **style** between two images, $X$ and $YS, as:

$$ D_S^L(X,Y) = ||G_{XL} - G_{YL}||^2 = \sum_{k, l}(G_{XL}(k, l) - G_{YL}(k, l))^2 $$

### Loss function

We define the total loss (content & style) as

$$ \$$

